In [1]:
"""
plan

1 import 
2 unpack data
21 collect paths for data
22 split data for train and test
3 create augmentation class iaa
4 create data generator 
5 create vgg-16 model
51 optimizer, early stopping, checkpoint
6 fit model
7 predict

"""

'\nplan\n\n1 import \n2 unpack data\n21 collect paths for data\n22 split data for train and test\n3 create augmentation class iaa\n4 create data generator \n5 create vgg-16 model\n51 optimizer, early stopping, checkpoint\n6 fit model\n7 predict\n\n'

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import cv2
import h5py
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D, GlobalMaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.metrics import categorical_crossentropy
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import imgaug.augmenters as iaa
%matplotlib inline

In [2]:
data_path = 'flowers'

data = []
rec = {}

for idx, directory in enumerate(os.listdir(data_path)):
    folder_path = data_path + '/' + directory
    rec[idx] = directory
    rec[directory] = idx
    for file in os.listdir(folder_path):
        file_path = folder_path + '/' + file
        data.append((file_path, idx))
        
data = pd.DataFrame(data, columns=['image', 'label'], index=None)

print(data.head)

<bound method NDFrame.head of                                            image  label
0       flowers/daisy/100080576_f52e8ee070_n.jpg      0
1       flowers/daisy/10140303196_b88d3d6cec.jpg      0
2     flowers/daisy/10172379554_b296050f82_n.jpg      0
3       flowers/daisy/10172567486_2748826a8b.jpg      0
4     flowers/daisy/10172636503_21bededa75_n.jpg      0
...                                          ...    ...
4312   flowers/tulip/9831362123_5aac525a99_n.jpg      4
4313   flowers/tulip/9870557734_88eb3b9e3b_n.jpg      4
4314   flowers/tulip/9947374414_fdf1d0861c_n.jpg      4
4315   flowers/tulip/9947385346_3a8cacea02_n.jpg      4
4316     flowers/tulip/9976515506_d496c5e72c.jpg      4

[4317 rows x 2 columns]>


In [3]:
train_data, test = train_test_split(data, test_size=0.1)
train_data = train_data.reset_index(drop=True)
test = test.reset_index(drop=True)
X_test, y_test = [], []

for i in range(len(test)):
    file_path = test.image.loc[i]
    label = test.label.loc[i]
    image = cv2.imread(file_path)
    image = cv2.resize(image, (224, 224))
    if image.shape[2] == 1:
        image = np.dstack([image, image, image])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.astype(np.float32) / 255.0
    X_test.append(image)
    y_test.append(to_categorical(label, num_classes=5))
    
X_test, y_test = np.array(X_test), np.array(y_test)
print("shape of X_test array -", X_test.shape)
print("shape of y_test array -", y_test.shape)
print("size of X_test array -", sys.getsizeof(X_test) // 10 ** 6, "MB")

shape of X_test array - (432, 224, 224, 3)
shape of y_test array - (432, 5)
size of X_test array - 260 MB


In [4]:
# Augmentation sequence 
seq = iaa.OneOf([
    iaa.Fliplr(), # horizontal flips
    iaa.Affine(rotate=20), # roatation
    iaa.Multiply((1.2, 1.5))]) #random brightness

In [5]:
def data_gen(data, batch_size):
    n = len(data)
    steps = n // batch_size
    
    batch_data = np.zeros((batch_size, 224, 224, 3), dtype=np.float32)
    batch_labels = np.zeros((batch_size, 5), dtype=np.float32)
    indices = np.arange(n)
    
    i = 0
    while True:
        counter = 0
        np.random.shuffle(indices)
        next_batch = indices[(batch_size*i):(batch_size*(i+1))]
        
        for idx in next_batch:
            image_name = data.image.loc[idx]
            label = data.label.loc[idx]
            label = to_categorical(label, num_classes=5)
            image = cv2.imread(image_name)
            image = cv2.resize(image, (224, 224))
            if image.shape[2] == 1:
                image = np.dstack([image, image, image])
            orig_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            orig_image = orig_image.astype(np.float32) / 255.0
            
            batch_data[counter] = orig_image
            batch_labels[counter] = label
            counter += 1
                
            if batch_size == counter:
                break
        
        i += 1
        yield batch_data, batch_labels
        
        if i >= steps:
            i = 0

In [6]:
def create_model():
    model = Sequential()

    # layer 1-2: 2 convolutional layers + 1 max-pooling layer 
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu', input_shape = (224,224,3)))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2,2), strides = 2))

    # number of filters and convolutions in each layer:
    filters_convs = [(128, 2), (256, 3), (512, 3), (512,3)]

    for n_filters, n_convs in filters_convs:
        for _ in np.arange(n_convs):
            model.add(Conv2D(filters = n_filters, kernel_size = (3,3), padding = 'same', activation = 'relu'))
    # max-pooling layer 
        model.add(MaxPooling2D(pool_size = (2,2), strides = 2))

    model.add(Flatten())
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5,activation = 'softmax'))

    # compile the model with a loss function, a metric and an optimization method
    opt = SGD(lr = 0.01) # stochastic gradient descent method with learning rate lr = 0.01
    model.compile(loss = categorical_crossentropy, 
                optimizer = opt, 
                metrics = ['accuracy'])
    
    return model

In [7]:
model = create_model()

"""f = h5py.File('vgg-16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r')

w,b = f['block1_conv1']['block1_conv1_W_1:0'], f['block1_conv1']['block1_conv1_b_1:0']
model.layers[1].set_weights = [w,b]

w,b = f['block1_conv2']['block1_conv2_W_1:0'], f['block1_conv2']['block1_conv2_b_1:0']
model.layers[2].set_weights = [w,b]

w,b = f['block2_conv1']['block2_conv1_W_1:0'], f['block2_conv1']['block2_conv1_b_1:0']
model.layers[4].set_weights = [w,b]

w,b = f['block2_conv2']['block2_conv2_W_1:0'], f['block2_conv2']['block2_conv2_b_1:0']
model.layers[5].set_weights = [w,b]

f.close()"""
model.summary()    

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

c:\users\skoro\appdata\local\programs\python\python39\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [8]:
es = EarlyStopping(patience=5)
checkpoint = ModelCheckpoint(filepath='model/best_model_todate', save_best_only=True, save_weights_only=True)

In [9]:
batch_size = 16
nb_epochs = 20

train_data_gen = data_gen(data=train_data, batch_size=batch_size)

nb_train_steps = train_data.shape[0]//batch_size

print("Number of training and validation steps: {} and {}".format(nb_train_steps, len(X_test)))

Number of training and validation steps: 242 and 432


In [ ]:
history = model.fit(train_data_gen, epochs=nb_epochs, steps_per_epoch=nb_train_steps,
                               validation_data=(X_test, y_test),callbacks=[es, checkpoint])

Epoch 1/20
201/242 [=======================>......] - ETA: 51s - loss: 1.6019 - accuracy: 0.2484